In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from langgraph.graph import StateGraph,START,END

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [2]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [3]:

def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [4]:

def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [5]:
graph = StateGraph(BlogState)

graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

graph.add_edge(START,'create_outline')
graph.add_edge('create_blog','create_outline')
graph.add_edge('create_outline',END)

workflow = graph.compile()

In [6]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': '## Blog Outline: Rise of AI in India\n\n**I. Introduction: Setting the Stage**\n\n   *  A. Hook: Start with a compelling statistic or anecdote about AI\'s impact in India. (e.g., Projected economic impact, successful AI startup story, government initiative).\n   *  B. Brief Definition of AI:  Explain AI in simple terms, avoiding technical jargon. Focus on its capabilities (learning, problem-solving, decision-making).\n   *  C. Thesis Statement:  Clearly state the blog\'s main argument, highlighting the significant rise of AI in India and its potential transformative impact across various sectors.  (e.g., "India is experiencing a significant surge in AI adoption, poised to revolutionize its economy and society through advancements in [mention key sectors].")\n   *  D.  Roadmap: Briefly outline the topics that will be covered in the blog.\n\n**II.  The Drivers of AI Growth in India: Why Now?**\n\n   *  A.  Government Initiatives and Policies:\

In [7]:
print(final_state['outline'])

## Blog Outline: Rise of AI in India

**I. Introduction: Setting the Stage**

   *  A. Hook: Start with a compelling statistic or anecdote about AI's impact in India. (e.g., Projected economic impact, successful AI startup story, government initiative).
   *  B. Brief Definition of AI:  Explain AI in simple terms, avoiding technical jargon. Focus on its capabilities (learning, problem-solving, decision-making).
   *  C. Thesis Statement:  Clearly state the blog's main argument, highlighting the significant rise of AI in India and its potential transformative impact across various sectors.  (e.g., "India is experiencing a significant surge in AI adoption, poised to revolutionize its economy and society through advancements in [mention key sectors].")
   *  D.  Roadmap: Briefly outline the topics that will be covered in the blog.

**II.  The Drivers of AI Growth in India: Why Now?**

   *  A.  Government Initiatives and Policies:
        *  1. National Strategy for Artificial Intelligenc